 # projeto sem objetivo pre definido

* primeiro fazer coleta dos dados para armazenamento 
*  sera feito coleta atraves de web scraping
* site: https://www.zapimoveis.com.br
* limpeza de dados
* talvez crie um banco de dados para mostrar a criacao de um banco de dados
* talvez pode ser feito uma coleta diaria durante um tempo para saber qto tempo um imovel fica disponivel para venda, qual o tempo medio de espera para ser vendido, qual o preço medio dos  imoveis mais vendidos 

apos coleta sera feita uma analise exploratoria para saber o que sera possivel de analisar, como por exemplo, valor media das casa, valor medio por regiao, a media de comodos, regioes  e comodos afetam os preços, qual tem mais impacto...?

tecnologia esperadas q serao usadas

* python
    * selenium ou beutiful soup para coleta de dados
    * pandas para exploraçao e limpeza

* power bi
    * para apresentaçao final

*github para fazer os historicos de evoluçao diaria e ter noçao de qto tempo levou para fazer o projeto (nao estou me obrigando a ter prazo, tenho outro trabalho entao sera feito nas horas disponiveis sem afetar outros compromissos)






In [11]:
#importando as bibliotecas necessarias
import pandas as pd
import selenium 
from selenium import webdriver
from selenium.webdriver.common.by import By
from time import sleep

In [192]:
# explarando o site com filtro de casa para venda e na ordem de maiores valores
driver_casa = webdriver.Chrome()
driver_casa.implicitly_wait(10)
driver_casa.get('https://www.zapimoveis.com.br/venda/casas/sp+sao-jose-dos-campos/?onde=,S%C3%A3o%20Paulo,S%C3%A3o%20Jos%C3%A9%20dos%20Campos,,,,,CITY,BR%3ESao%20Paulo%3ENULL%3ESao%20Jose%20dos%20Campos,-23.21984,-45.891566,&ordem=Valor&pagina=1&tipo=Im%C3%B3vel%20usado&tipoUnidade=Residencial,Casa&transacao=Venda')
driver_casa.maximize_window()
original_window = driver_casa.current_window_handle

In [193]:
#pegando os anuncios das casas
anuncios = driver_casa.find_elements(By.CLASS_NAME, 'card-container')

In [194]:
#conferindo a quantidade de elementos 
len(anuncios)

24

entrando no anuncio para verificar o que é possivel extrair 

fazendo uma breve verificacao em algums anuncio vi q os valores nao correspondem ao valor real,
 entao apos a coleta de dados sera necessario ainda uma limpeza 

 aqui ja da para extrair :

 valor do imovel
 
 condominio
 
 iptu
 
 area
 
 quantidade de banheiro
 
 quantidade de quartos
 
 quantidade de vagas na garagem
 
 rua, numero e bairro (cidade e estado sao sempre iguais, ja q é uma pesquisa de imoveis de sao jose dos campos)
 
 se esta por imobiliaria ou nao
 
 extrair tambem as informaçoes das descriçoes para complentar alguma informaçao
 
 a descricao tbm pode estar como caracteristica
 
 quantidade de andares
    

In [277]:
# window_handles eh uma lista de todas as guias q estao abertas, usar esse for pra saber quais os nomes das guias
for window_handle in driver_casa.window_handles:
    print(window_handle)

CDwindow-4847DB41A0E310EE4DB0A5418C1BA2E9


In [278]:
original_driver = driver_casa.current_window_handle
original_driver

'CDwindow-4847DB41A0E310EE4DB0A5418C1BA2E9'

In [276]:
driver_casa.switch_to.window(driver_casa.window_handles[0])

In [ ]:
dados = pd.DataFrame(columns= ['Endereço', 'Preço', 'Condominio', 'Iptu', 'Lista Comodos','Lista Caracteristicas', 'Obserevacoes'])

for i in range(len(anuncios)):
    while len(driver_casa.window_handles) == 1:
        driver_casa.find_elements(By.CLASS_NAME, 'card-container')[i].click()
        sleep(2)
        if len(driver_casa.window_handles) > 1 :
            driver_casa.switch_to.window(driver_casa.window_handles[1])
            sleep(2)
            break

    

    endereco= driver_casa.find_element(By.CLASS_NAME, 'info__map-link ')   

    preco = driver_casa.find_element(By.CLASS_NAME, 'price__item--main')

    condominio = driver_casa.find_elements(By.CLASS_NAME, 'price__item')[0]

    iptu = driver_casa.find_elements(By.CLASS_NAME, 'price__item')[1]

    comodos = driver_casa.find_elements(By.CLASS_NAME, 'feature__item')
    lista_comodos = [comodo.text for comodo in comodos]

    caracteristicas = driver_casa.find_elements(By.CLASS_NAME, 'amenities__list-item')
    lista_caracteristicas = [caracteristica.text for caracteristica in caracteristicas]

    obserevacoes = driver_casa.find_element(By.CLASS_NAME, 'amenities__base-description')
    
    
    #print()
    #print()
    #print(f'indice ===={i}')
    #print(driver_casa.current_window_handle)
    #print()
    #print(f'Endereço = {endereco.text}')
    #print(f'preco = {preco.text}')
    #print(f'condominio = {condominio.text}')
    #print(f'iptu = {iptu.text}')
    #print(f'Area = {lista_comodos[4]}')
    #print(f'Quartos = {lista_comodos[5]}')
    #print(f'Banheiros = {lista_comodos[-1]}')
    #print(f'Caracteristica = {caracteristica}')
    #print(f'obserevacoes = {obserevacoes.text}')
    #print()
    #print()
    
    
    caracteristica = [caracteristica for caracteristica in caracteristicas]
    dic_dados = {'Endereço':[endereco.text],
             'Preço' : [preco.text],
             'Condominio': [condominio.text],
             'Iptu': [iptu.text],
             'Lista Comodos': lista_comodos,
             'Lista Caracteristicas' : lista_caracteristicas,
             'Obserevacoes' : [obserevacoes.text]}
    
    
    while len(driver_casa.window_handles) > 1:
        driver_casa.switch_to.window(driver_casa.window_handles[1])
        sleep(2)
        driver_casa.close()
        sleep(2)
        driver_casa.switch_to.window(original_driver)
        sleep(2)
    
    print()
    print()
    print(f'indice ========== {i}')
    print(dic_dados)
    print()
    print()
    dados = dados.append(dic_dados, ignore_index = True)
dados